In [20]:
import sys, os, random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras.backend.tensorflow_backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import normalize
%matplotlib inline

## Prepare fc1000 engineered images

In [2]:
root = "/home/ssnazrul/WIFIRE/ResNet50_Analysis/WIFIRE_Engineered/"

image_features = pd.read_csv(root+"WIFIRE_fc1000_engineered_features.csv",  delimiter=',', header=None)
df = pd.read_csv(root+'WIFIRE_labels.csv')
labels = df.Labels.values

In [3]:
image_features = image_features.values #convert df to np array

In [39]:
np.shape(image_features)

(437153, 1000)

## Keras Modelling

Create Model

In [40]:
def baseline_model(device = 0):
    #Shut down tensorflow after use
    with K.tf.device('/gpu:'+str(device)):
        # create model
        model = Sequential()
        model.add(Dense(12, input_shape=(1000,)))
        # Compile model
        model.compile(loss='mse', optimizer='adam')
        return model

In [34]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [17]:
X = normalize(image_features)
Y = labels

In [7]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [31]:
kfold = KFold(n_splits=2, shuffle=True)

In [42]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

ValueError: Error when checking target: expected dense_6 to have shape (None, 1000) but got array with shape (218576, 12)